#  **SWT 2024 Evaluating your Machine Learning Models in Snowflake**
### Notebook 1 - Data Ingestion
---
### What We'll Do:
1. **Data Ingestion**: Fetch customer and claims data from our database
2. **Data Transformation**: Utilize Snowpark DataFrames for data preparation and analysis
3. **Model Training**: Train a XGB Classifier model
4. **Model Registry**: Saving the model to Snowflake Model Registry

Remember to add the necessary packages in the 'Packages' drop down at the top. For example,
- `snowflake-snowpark-python`
- `snowflake-ml-python`
- `pandas`
- etc.

In [ ]:
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)
from snowflake.snowpark import Window
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import LongType
from snowflake.ml.modeling.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.model_selection import GridSearchCV
from snowflake.ml.modeling.metrics import *
from snowflake.ml.registry import Registry

# Import python packages
import streamlit as st
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import json
import tabulate
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
snowflake_environment = session.sql('select current_user(), current_version()').collect()
from snowflake.snowpark.version import VERSION
from snowflake.ml import version

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(VERSION[0],VERSION[1],VERSION[2]))
print('Snowflake ML version        : {}.{}.{}'.format(version.VERSION[0],version.VERSION[2],version.VERSION[4]))

In [ ]:
claim_data = session.read.table("SWT2024_DEMO_AUTO_INSURANCE.DATA.CLAIM_DATA")
customer_data = session.read.table("SWT2024_DEMO_AUTO_INSURANCE.DATA.CUSTOMER_DATA")

st.dataframe(claim_data.limit(50))
st.dataframe(customer_data.limit(50))

In [ ]:
claim_data = claim_data.replace('?', None)
claim_data.filter(col("POLICE_REPORT_AVAILABLE").is_null()).show(10)
# Calculate the mode of the 'POLICE_REPORT_AVAILABLE' column
mode_value = claim_data.select(mode(col("POLICE_REPORT_AVAILABLE"))).collect()[0][0]
print(f"Fill NULL value in POLICY_REPORT_AVAIALLBE to the mode: {mode_value}")
# Fill NULL values with the mode
claim_data = claim_data.with_column("POLICE_REPORT_AVAILABLE", 
    when(col("POLICE_REPORT_AVAILABLE").is_null(), mode_value)
    .otherwise(col("POLICE_REPORT_AVAILABLE")))
claim_data.filter(col("POLICE_REPORT_AVAILABLE").is_null()).show(10)

In [ ]:
database = 'SWT2024_DEMO_AUTO_INSURANCE'
schema = 'DATA'
warehouse = 'DEMO_WH'

fs = FeatureStore(
    session=session, 
    database=database,
    name=schema,
    default_warehouse=warehouse,
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

In [ ]:
# Snowflake Feature Store requires an "entity" with "join_keys" be registered
POLICY_NUMBER = Entity(name="POLICY_NUMBER", join_keys=["POLICY_NUMBER"])
fs.register_entity(POLICY_NUMBER)
fs.list_entities().show()

In [ ]:
from snowflake.cortex import Complete


llm = 'llama3-70b'

prompt = f"""
Given the SQL Code for selecting the dataframe: {customer_data.queries['queries'][0]}
A sample of the dataframe: {customer_data.sample(n=30).to_pandas().to_markdown()}. 
Describe the following features of the data given succinctly: ['AGE', 'POLICY_START_DATE', 'POLICY_LENGTH_MONTH', 'POLICY_DEDUCTABLE', 'POLICY_ANNUAL_PREMIUM', 'INSURED_SEX', 'INSURED_EDUCATION_LEVEL', 'INSURED_OCCUPATION'] 
For context, this is customer and policy information about individuals that have insurance with the company.
The descriptions will be stored in a feature store in Snowflake. Return a JSON where the feature name is the key and the description is the value.
"""
llm_response = Complete(llm, prompt)

feature_desc = json.loads(llm_response.split('```')[1])
for key in feature_desc:
    feature_desc[key] = feature_desc[key].replace("'", '')
feature_desc

In [ ]:
fv = FeatureView(
    name="customer_data",
    entities = [POLICY_NUMBER],
    feature_df = customer_data,
    #refresh_freq="1 hour",  # can also be a cron schedule - * * * * * America/Los_Angeles
    desc="Insurance Customer Data")

fv = fv.attach_feature_desc(feature_desc)

#Let's register this FeatureView in Snowflake 
registered_fv = fs.register_feature_view(
    feature_view=fv,
    version="V1",
    overwrite = True
)

In [ ]:
# fv = FeatureView(
#    name="customer_data",
#    entities = [POLICY_NUMBER],
#    feature_df=customer_data,    
# )
# registered_fv = fs.register_feature_view(
#    feature_view=fv,
#    version="V2"
# )
# fs.delete_feature_view(registered_fv)

In [ ]:
customer_fv = fs.get_feature_view(
    name = 'customer_data',
    version = 'V1'
)

training_data = fs.generate_dataset(
    name="Harleytest",
    spine_df=claim_data,
    features=[customer_fv],
    spine_label_cols = ["FRAUD_REPORTED"]
)

training_data_df = training_data.read.to_snowpark_dataframe()

In [ ]:
# Understand the policy duration from the policy start date to the indicent date
from snowflake.snowpark.functions import col

training_data_df = training_data_df.with_column("POLICY_DURATION",
    floor(datediff("month", col("POLICY_START_DATE"), col("INCIDENT_DATE"))))

In [ ]:
# Convert Snowpark DataFrame to pandas DataFrame
pandas_df = training_data_df.to_pandas()

# Select only numeric columns
numeric_columns = pandas_df.select_dtypes(include=['int64', 'float64']).columns
numeric_df = pandas_df[numeric_columns]

# Calculate correlation matrix
corr_matrix = numeric_df.corr()
sns.set(font_scale=0.5)
# Create heatmap
plt.figure(figsize=(7, 5))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", vmin=-1, vmax=1, center=0)
plt.title("Correlation Heatmap of Claim Data", fontdict = {'fontsize' : 12})

In [ ]:
# Due to the high correlation between Age and policy_length_month, let's drop age.
# Let's all drop the date fields 
training_data_df = training_data_df.drop("age", "INCIDENT_DATE", "POLICY_START_DATE")

In [ ]:
# Now, we want to start to encode all categorical variable and turning all the strings into numeric fields so our model can train on them
# Select only string columns (equivalent to 'object' dtype in pandas)
categorical_claim_data = training_data_df.select([col for col in training_data_df.columns if isinstance(training_data_df.schema[col].datatype, StringType)])

# Print unique values for each column
for col in categorical_claim_data.columns:
    unique_values = categorical_claim_data.select(col).distinct().collect()
    unique_list = [row[col] for row in unique_values]
    print(f"{col}:")
    print(unique_list)
    print()  # Add a blank line for readability

In [ ]:
from snowflake.snowpark.functions import col

training_data_df = training_data_df.with_column("FRAUD_REPORTED", col("FRAUD_REPORTED").astype(LongType()))
train_data, test_data = training_data_df.random_split(weights = [0.8, 0.2], seed = 42)

In [ ]:
train_data.write.save_as_table('SWT2024_DEMO_AUTO_INSURANCE.DATA.TRAIN_DATA', mode = 'overwrite')
test_data.write.save_as_table('SWT2024_DEMO_AUTO_INSURANCE.DATA.TRAIN_DATA', mode = 'overwrite')

In [ ]:
st.dataframe(train_data.limit(50))

In [ ]:
# Define the categories with their specific order
categories = {
    "INSURED_EDUCATION_LEVEL": np.array(["High School", "Associate", "College", "Masters", "JD", "MD", "PhD"]),
    "INCIDENT_SEVERITY": np.array(["Trivial Damage", "Minor Damage", "Major Damage", "Total Loss"])
}
# Create the OrdinalEncoder with specified categories
OrdinalEncoding = OrdinalEncoder(
    input_cols=["INSURED_EDUCATION_LEVEL", "INCIDENT_SEVERITY"],
    output_cols=["INSURED_EDUCATION_LEVEL_OE", "INCIDENT_SEVERITY_OE"],
    categories=categories,
    handle_unknown="use_encoded_value",
    unknown_value=-1,
    drop_input_cols=True
)

# Define the columns to encode
columns_to_encode = [
    "INSURED_SEX",
    "INSURED_OCCUPATION",
    "INCIDENT_TYPE",
    "AUTHORITIES_CONTACTED",
    "POLICE_REPORT_AVAILABLE"
]
# Create a OneHotEncoder instance
OneHotEncoding = OneHotEncoder(
    input_cols=columns_to_encode,
    output_cols=[f"{col}_encoded" for col in columns_to_encode],
    drop_input_cols=True,  # Keep original columns
    handle_unknown='ignore'  # Ignore any unknown categories during transform
)

# Define the columns to scale
columns_to_scale = [
    'POLICY_LENGTH_MONTH',
    'POLICY_DEDUCTABLE',
    'POLICY_ANNUAL_PREMIUM',
    'CLAIM_AMOUNT',
    'POLICY_DURATION'
]
# Create the StandardScaler
StandardScaling = StandardScaler(
    input_cols=columns_to_scale,
    output_cols=[f"{col}_SCALED" for col in columns_to_scale],
    with_mean=True,
    with_std=True,
    drop_input_cols=True  # Keep original columns
)

# Determine the label column name
# feature_columns = train_data.columns.remove('FRAUD_REPORTED_LONG')
label_column = ['FRAUD_REPORTED']
output_column = ['PREDICTED_FRAUD']


# # Initially, we can run this under the XGB Classifier model. However, you will notice that
# # the model overfits on the training data and performs poorly on the test dataset
# xgbmodel = XGBClassifier(
#     random_state=1, 
#     #input_cols=feature_columns,    #here we are passing all columns so we have commented out. If you have specific columns set as features, you should specify them here
#     label_cols=label_column,
#     output_cols=output_column
#     )


xgb_grid_search = GridSearchCV(
    estimator=XGBClassifier(),
    param_grid={
        "n_estimators":[10, 20, 30, 50, 100, 150, 200, 250, 300],
        "subsample": [0.9, 0.5, 0.2],
        "max_depth": range(2,10,1),
        "learning_rate":[0.1, 0.06, 0.05, 0.03, 0.01, 0.005, 0.002, 0.001],
    },
    n_jobs = -1,
    #input_cols=feature_columns,    #here we are passing all columns so we have commented out. 
                                    #If you have specific columns set as features, you should specify them here
    label_cols=label_column,
    output_cols=output_column,
)

# xgb_gs_fitted = xgb_grid_search.fit(train_data)

model_pipeline = Pipeline(
    steps=[
        ("Ordinal_encoding",OrdinalEncoding),
        ("OneHotEncoding",OneHotEncoding),
        ("standardscaler",StandardScaling),
        #("XGBClassifier", xgbmodel)
        ("CV_XGBClassifier", xgb_grid_search)
    ]
)

In [ ]:
wh = str(session.get_current_warehouse()).strip('"')
print(f"Current warehouse: {wh}")
print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

session.sql(f"alter warehouse {session.get_current_warehouse()} set WAREHOUSE_SIZE = LARGE").collect()

#Give Snowflake a few seconds to change WH sizes
import time
time.sleep(5)

print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

In [ ]:
# Fit the pipeline to the training data
xgb_gs_fitted_training = model_pipeline.fit(train_data)

In [ ]:
wh = str(session.get_current_warehouse()).strip('"')
print(f"Current warehouse: {wh}")
print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

session.sql(f"alter warehouse {session.get_current_warehouse()} set WAREHOUSE_SIZE = SMALL").collect()

#Give Snowflake a few seconds to change WH sizes
import time
time.sleep(5)

print(session.sql(f"SHOW WAREHOUSES LIKE '{wh}';").collect())

In [ ]:
gs_results = xgb_gs_fitted_training.to_sklearn().named_steps['CV_XGBClassifier'].cv_results_
n_estimators_val = []
learning_rate_val = []
for param_dict in gs_results["params"]:
    n_estimators_val.append(param_dict["n_estimators"])
    learning_rate_val.append(param_dict["learning_rate"])
mape_val = gs_results["mean_test_score"]

gs_results_df = pd.DataFrame(data={
    "n_estimators":n_estimators_val,
    "learning_rate":learning_rate_val,
    "mape":mape_val})

sns.set_context("notebook", font_scale=0.5)
sns.relplot(data=gs_results_df, x="learning_rate", y="mape", hue="n_estimators", kind="line", height=3)
plt.show()

In [ ]:
xgb_gs_train = model_pipeline.predict(train_data)
xgb_gs_predictions = model_pipeline.predict(test_data)

In [ ]:
# Let's start with the basic metric, Accuracy, which the number of correct predictions made divided by the total number of predictions made,
ACCURACY = accuracy_score(df=xgb_gs_predictions, y_true_col_names=label_column, y_pred_col_names=output_column)
print('Training Accuracy:', accuracy_score(df=xgb_gs_train, y_true_col_names=label_column, y_pred_col_names=output_column))
print(f'Test Acccuracy: {ACCURACY}')

# RPC AUC is slightly perferred IMO. Anything above 50% or .5 is better than random guessing
AUC = roc_auc_score(df=xgb_gs_predictions, y_true_col_names=label_column, y_score_col_names=output_column)
print('Training AUC:', roc_auc_score(df=xgb_gs_train, y_true_col_names=label_column, y_score_col_names=output_column))
print(f'Test AUC: {AUC}')

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].best_estimator_)
print("\n The best score across ALL searched params:\n",model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].best_score_)
print("\n The best parameters across ALL searched params:\n",model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].best_params_)

In [ ]:
# Plot feature importance
feat_importance = pd.DataFrame(model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].best_estimator_.feature_importances_,model_pipeline.to_sklearn().named_steps['CV_XGBClassifier'].feature_names_in_,columns=['FeatImportance'])
feat_importance.sort_values('FeatImportance').plot.barh(y='FeatImportance', figsize=(5,10))

In [ ]:
# Let's now register the CV Classfier model into the model_registry
Reg = Registry(
    session=session,
    database_name=session.get_current_database(),
    schema_name='data',
)

In [ ]:
# FUNCTION used to iterate the model version so we can automatically 
# create the next version number
import ast
import builtins  # Import the builtins module
#from snowflake.snowpark import functions as F 

def get_next_version(reg, model_name) -> str:
    """
    Returns the next version of a model based on the existing versions in the registry.

    Args:
        reg: The registry object that provides access to the models.
        model_name: The name of the model.

    Returns:
        str: The next version of the model in the format "V_".

    Raises:
        ValueError: If the version list for the model is empty or if the version format is invalid.
    """
    models = reg.show_models()
    if models.empty:
        return "V_1"
    elif model_name not in models["name"].to_list():
        return "V_1"
    max_version_number = builtins.max(  
        [
            int(version.split("_")[-1])
            for version in ast.literal_eval(
                models.loc[models["name"] == model_name, "versions"].values[0]
            )
        ]
    )
    return f"V_{max_version_number + 1}"

In [ ]:
model_name = 'XGB_GS_FRAUD_MODEL'
model_version = get_next_version(Reg, model_name)

mv = Reg.log_model(xgb_gs_fitted_training,
    model_name=model_name,
    version_name=model_version,
    conda_dependencies=["snowflake-ml-python"],
    comment="Model trained using GridsearchCV in Snowpark to predict fraud claims",
    #metrics={"Acc": ACCURACY, "AUC": AUC}, # We can save our model metrics here
    options= {"relax_version": False}
)

m = Reg.get_model(model_name)
m.default = model_version

In [ ]:
# lets see the models we have in our registry

Reg.get_model(model_name).show_versions()

In [ ]:
st.dataframe(xgb_gs_predictions.limit(20))
#xgb_gs_predictions.sample(n=200).write.save_as_table('SWT2024_DEMO_AUTO_INSURANCE.DATA.REFERENCE_DATA', mode = 'overwrite')